In [1]:
import numpy as np
import re
import open3d as o3d
from skimage import measure
import os
from utils import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [112]:
VOXEL_GRID_SIZE = 256

In [113]:
files = [f for f in os.listdir("data") if f.endswith('.x')]

In [114]:
meshes = []
for f in files:
    print(f)
    filepath = os.path.abspath(f'data\\{f}')
    raw_data = parse_x_file(filepath)

    # Переведём координаты точек из локальной системы координат в систему координат камеры
    # world_transform = apply_transformation(raw_data.get('vertices'), raw_data.get('frame_matrix')) # Даёт худшие результаты
    world_transform = raw_data.get('vertices')
    transformed = apply_transformation(world_transform, raw_data.get('rv_matrix'))
    meshes.append(transformed)

teapot_1.x
num_vertices: 200310
num_faces: 379304
num_uvs: 200310
teapot_2.x
num_vertices: 190307
num_faces: 358169
num_uvs: 190307


In [ ]:
# Чтобы получить больше информации о процессе объединения мешей
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)

In [ ]:
pcd = merge_meshes(meshes, VOXEL_GRID_SIZE, mcd_coarse_scale=30, mcd_fine_scale=7)

In [ ]:
# Считаем размер одного вокселя
_, _, voxel_size, _ = get_mesh_params(meshes, VOXEL_GRID_SIZE)

In [472]:
# Задаём параметры для объединения мешей
max_correspondence_distance_coarse = voxel_size * 30
max_correspondence_distance_fine = voxel_size * 7

In [ ]:
# Конфертируем все меши в PointCloud
pcds_raw = [vertices_to_pcd(m) for m in meshes]
# Считаем нормали для PointCloud
pcds_down = [get_normals_to_pcd(pcd) for pcd in pcds_raw]

In [ ]:
# Посмотрим, как взаимнрасположены меши
o3d.visualization.draw_geometries(pcds_down)

In [ ]:
# Находим пересечения PointCloud
pose_graph = full_registration(pcds_down,
                               max_correspondence_distance_coarse,
                               max_correspondence_distance_fine)

In [ ]:
# Подбираем оптимальные параметры транформации для объединения
option = o3d.pipelines.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.25,
    reference_node=0)
o3d.pipelines.registration.global_optimization(
    pose_graph, o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
    o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(), option)

In [ ]:
# Трансформируем все PointCloud
for point_id in range(len(pcds_down)):
    print(pose_graph.nodes[point_id].pose)
    pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)

In [ ]:
# Посмотрим на трансформированные PointCloud
o3d.visualization.draw_geometries(pcds_down)

In [ ]:
# Объединяем точки в один PointCloud
pcds_raw = [vertices_to_pcd(m) for m in meshes]
pcds = [get_normals_to_pcd(pcd) for pcd in pcds_raw]
pcd_combined = o3d.pybind.geometry.PointCloud()
for point_id in range(len(pcds)):
    pcds[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds[point_id]
pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=float(voxel_size))

In [ ]:
# Пересчитаем нормали объединённого PointCloud
pcd_norm = get_normals_to_pcd(pcd_combined_down)

In [ ]:
# Посмотрим на результат объединения
o3d.visualization.draw_geometries([pcd_norm])

In [106]:
# Построим воксельную структуру из откалиброванных точек всех мешей
voxels, origin, _ = build_voxel_grid(meshes, grid_size=VOXEL_GRID_SIZE)

In [107]:
# Построим меш из воксельной струкуктуры
mesh = mesh_from_voxels(voxels, voxel_size=voxel_size, origin=origin)

In [108]:
mesh

TriangleMesh with 196026 points and 413452 triangles.

In [109]:
o3d.visualization.draw_geometries([mesh])

Full registration ...
[Open3D DEBUG] ICP Iteration #0: Fitness 0.9198, RMSE 7.7567
[Open3D DEBUG] ICP Iteration #1: Fitness 0.9249, RMSE 7.1981
[Open3D DEBUG] ICP Iteration #2: Fitness 0.9299, RMSE 6.9502
[Open3D DEBUG] ICP Iteration #3: Fitness 0.9323, RMSE 6.7491
[Open3D DEBUG] ICP Iteration #4: Fitness 0.9337, RMSE 6.6078
[Open3D DEBUG] ICP Iteration #5: Fitness 0.9348, RMSE 6.5126
[Open3D DEBUG] ICP Iteration #6: Fitness 0.9359, RMSE 6.4619
[Open3D DEBUG] ICP Iteration #7: Fitness 0.9367, RMSE 6.4281
[Open3D DEBUG] ICP Iteration #8: Fitness 0.9374, RMSE 6.4043
[Open3D DEBUG] ICP Iteration #9: Fitness 0.9379, RMSE 6.3891
[Open3D DEBUG] ICP Iteration #10: Fitness 0.9383, RMSE 6.3768
[Open3D DEBUG] ICP Iteration #11: Fitness 0.9386, RMSE 6.3701
[Open3D DEBUG] ICP Iteration #12: Fitness 0.9389, RMSE 6.3663
[Open3D DEBUG] ICP Iteration #13: Fitness 0.9392, RMSE 6.3665
[Open3D DEBUG] ICP Iteration #14: Fitness 0.9394, RMSE 6.3622
[Open3D DEBUG] ICP Iteration #15: Fitness 0.9397, RMSE 6.3

Optimizing PoseGraph ...
[Open3D DEBUG] Validating PoseGraph - finished.
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 2 nodes and 1 edges.
[Open3D DEBUG] Line process weight : 5504017.244258
[Open3D DEBUG] [Initial     ] residual : 3.336700e-24, lambda : 9.775878e+03
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 2 nodes and 1 edges.
[Open3D DEBUG] Line process weight : 5504017.244258
[Open3D DEBUG] [Initial     ] residual : 3.336700e-24, lambda : 9.775878e+03
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] CompensateReferencePoseGraphNode : reference : 0


Transform points and display
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[  0.9558904   -0.24411807   0.16333983   5.71053283]
 [  0.2918158    0.72601631  -0.62269082 -11.95014514]
 [  0.0334227    0.64288932   0.76522953 -10.86034223]
 [  0.           0.           0.           1.        ]]
[Open3D DEBUG] [Visualizer] Creating window.
[Open3D DEBUG] GLFW init.
[Open3D DEBUG] Add geometry and update bounding box to [(-0.9998, -83.8568, -51.5618) - (123.1499, 119.4377, 49.6524)]
[Open3D DEBUG] Add geometry and update bounding box to [(-1.1539, -83.8568, -51.5618) - (125.4630, 120.0148, 51.6416)]
[Open3D DEBUG] [Visualizer] Destroying window.
[Open3D DEBUG] GLFW destruct.


Make a combined point cloud


[Open3D DEBUG] Pointcloud down sampled from 390617 points to 56946 points.
[Open3D DEBUG] Write geometry::PointCloud: 56946 vertices.
[Open3D DEBUG] [Visualizer] Creating window.
[Open3D DEBUG] GLFW init.
[Open3D DEBUG] Add geometry and update bounding box to [(-1.0762, -83.8250, -51.3693) - (125.3963, 119.9616, 51.7213)]
[Open3D DEBUG] [Visualizer] Destroying window.
[Open3D DEBUG] GLFW destruct.


[Open3D DEBUG] [Visualizer] Creating window.
[Open3D DEBUG] GLFW init.
[Open3D DEBUG] Add geometry and update bounding box to [(-1.0762, -83.8250, -51.3693) - (125.3963, 119.9616, 51.7213)]
[Open3D DEBUG] [Visualizer] Destroying window.
[Open3D DEBUG] GLFW destruct.
